<a href="https://colab.research.google.com/github/arkajyotiMukherjee/tensorflow_docs_prac/blob/master/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow==2.0.0-beta1

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version: ',tf.__version__)
print('Eger execution: ', tf.executing_eagerly())
print('Hub version: ', hub.__version__)
print('GPU is ', 'available' if tf.test.is_gpu_available() else 'Not Found')
tf.test.gpu_device_name()

Version:  2.0.0-beta1
Eger execution:  True
Hub version:  0.5.0
GPU is  Not Found


''

In [0]:
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])

(train_data, validation_data), test_data = tfds.load(name='imdb_reviews',
                                                    split=(train_validation_split, tfds.Split.TEST),
                                                    as_supervised=True)

In [4]:
train_data_batch, train_labels_batch = next(iter(train_data.batch(10)))
print(train_data_batch)
print(train_labels_batch)

tf.Tensor(
[b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novelist. It'

In [0]:
# Embedding layer
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], output_shape=[128], dtype=tf.string, trainable=True)

In [6]:
hub_layer(train_data_batch[:3])

<tf.Tensor: id=401, shape=(3, 128), dtype=float32, numpy=
array([[ 1.78428471e+00, -2.29856372e-01,  2.45664164e-01,
         6.84022367e-01, -1.29306912e-01, -1.57380581e-01,
        -1.81604568e-02, -3.73616189e-01, -1.86627433e-01,
         2.31135096e-02,  2.92425722e-01, -4.98053074e-01,
        -4.21140671e-01, -4.68299031e-01, -3.36686820e-01,
        -1.38489604e-01, -3.70901823e-01, -1.46785565e-02,
        -4.20534164e-01,  1.43151474e+00,  1.00796312e-01,
         4.05581176e-01, -8.53958651e-02, -1.26951560e-01,
        -1.08419672e-01, -3.50219727e-01,  4.06883471e-02,
        -1.95807740e-01, -1.16975710e-01, -5.78830671e-03,
         9.03800651e-02, -9.08162072e-02, -8.68640095e-03,
        -1.64070018e-02,  3.24384898e-01, -9.46734697e-02,
        -1.63240522e-01, -4.01422143e-01, -4.99349594e-01,
         5.78139901e-01, -1.41224876e-01,  3.35231982e-02,
        -1.43409833e-01, -1.45958960e-01,  4.64950085e-01,
         1.25476301e-01, -1.26240194e-01, -5.81108630e-01

In [7]:
# Build model
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=20,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/20


W0710 16:14:25.099171 140543819302784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 48s 2s/step - loss: 0.6374 - accuracy: 0.6660 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 47s 2s/step - loss: 0.4777 - accuracy: 0.8401 - val_loss: 0.4268 - val_accuracy: 0.8454
Epoch 3/20
30/30 [==============================] - 47s 2s/step - loss: 0.3162 - accuracy: 0.9027 - val_loss: 0.3349 - val_accuracy: 0.8662
Epoch 4/20
30/30 [==============================] - 47s 2s/step - loss: 0.2087 - accuracy: 0.9379 - val_loss: 0.2974 - val_accuracy: 0.8778
Epoch 5/20
30/30 [==============================] - 47s 2s/step - loss: 0.1378 - accuracy: 0.9628 - val_loss: 0.2879 - val_accuracy: 0.8776
Epoch 6/20
30/30 [==============================] - 47s 2s/step - loss: 0.0899 - accuracy: 0.9821 - val_loss: 0.2953 - val_accuracy: 0.8768
Epoch 7/20
30/30 [==============================] - 47s 2s/step - loss: 0.0588 - accuracy: 0.9913 - val_loss: 0.3075 - val_accuracy: 0.8746
Epoch 8/20
30/30 [=====

In [10]:
results = model.evaluate(test_data.batch(512))

for name, value in zip(model.metrics_names, results):
  print('\n%s: %.3f'%(name, value*100))

     49/Unknown - 10s 212ms/step - loss: 0.5148 - accuracy: 0.8524
loss: 51.475

accuracy: 85.244
